In [2]:
import ee

# Initialize the Earth Engine API.
ee.Authenticate(force = True)
ee.Initialize(project='ee-hadat')


Successfully saved authorization token.


In [3]:
import ee
import math

# Initialize Earth Engine (you need to run ee.Authenticate() and ee.Initialize() beforehand in your environment)
# ee.Authenticate()
# ee.Initialize()

def terrainCorrection(image):
    imgGeom = image.geometry()
    srtm = ee.Image('USGS/SRTMGL1_003').clip(imgGeom)  # 30m SRTM
    sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

    # 2.1.1 Radar geometry
    theta_i = image.select('angle')
    phi_i = ee.Terrain.aspect(theta_i)\
        .reduceRegion(ee.Reducer.mean(), theta_i.get('system:footprint'), 1000)\
        .get('aspect')

    # 2.1.2 Terrain geometry
    alpha_s = ee.Terrain.slope(srtm).select('slope')
    phi_s = ee.Terrain.aspect(srtm).select('aspect')

    # 2.1.3 Model geometry
    phi_r = ee.Image.constant(phi_i).subtract(phi_s)

    # Convert all to radians
    phi_rRad = phi_r.multiply(math.pi / 180)
    alpha_sRad = alpha_s.multiply(math.pi / 180)
    theta_iRad = theta_i.multiply(math.pi / 180)
    ninetyRad = ee.Image.constant(90).multiply(math.pi / 180)

    # Slope steepness in range (eq. 2)
    alpha_r = alpha_sRad.tan().multiply(phi_rRad.cos()).atan()

    # Slope steepness in azimuth (eq. 3)
    alpha_az = alpha_sRad.tan().multiply(phi_rRad.sin()).atan()

    # Local incidence angle (eq. 4)
    theta_lia = alpha_az.cos().multiply(theta_iRad.subtract(alpha_r).cos()).acos()
    theta_liaDeg = theta_lia.multiply(180 / math.pi)

    # 2.2 Gamma_nought_flat
    gamma0 = sigma0Pow.divide(theta_iRad.cos())
    gamma0dB = ee.Image.constant(10).multiply(gamma0.log10())
    ratio_1 = gamma0dB.select('VV').subtract(gamma0dB.select('VH'))

    # Volumetric Model
    nominator = ninetyRad.subtract(theta_iRad).add(alpha_r).tan()
    denominator = ninetyRad.subtract(theta_iRad).tan()
    volModel = nominator.divide(denominator).abs()

    # Apply model
    gamma0_Volume = gamma0.divide(volModel)
    gamma0_VolumeDB = ee.Image.constant(10).multiply(gamma0_Volume.log10())

    # Layover/shadow mask
    alpha_rDeg = alpha_r.multiply(180 / math.pi)
    layover = alpha_rDeg.lt(theta_i)
    shadow = theta_liaDeg.lt(85)

    # Calculate the ratio for RGB vis
    ratio = gamma0_VolumeDB.select('VV').subtract(gamma0_VolumeDB.select('VH'))

    output = gamma0_VolumeDB.addBands(ratio).addBands(alpha_r).addBands(phi_s).addBands(theta_iRad)\
        .addBands(layover).addBands(shadow).addBands(gamma0dB).addBands(ratio_1)

    return image.addBands(
        output.select(['VV', 'VH'], ['VV', 'VH']),
        None,
        True
    )


def powerToDb(img):
    return ee.Image(10).multiply(img.log10())


def dbToPower(img):
    return ee.Image(10).pow(img.divide(10))


def leeSigma(img):
    time = img.copyProperties(img, ['system:time_start'])
    bandNames = img.bandNames()

    pow = dbToPower(img)

    kernelWeights = ee.List.repeat(ee.List.repeat(1, 9), 9)
    kernel = ee.Kernel.fixed(9, 9, kernelWeights, 5, 5)

    targetWeights = ee.List.repeat(ee.List.repeat(1, 3), 3)
    targetKernel = ee.Kernel.fixed(3, 3, targetWeights, 1, 1)

    Tk = ee.Image(7)
    sigma = '0.9'
    looks = '4'

    # Lookup table for range and eta values for intensity
    sigmaLookup = ee.Dictionary({
        '1': ee.Dictionary({
            '0.5': ee.Dictionary({'A1': 0.436, 'A2': 1.92, 'η': 0.4057}),
            '0.6': ee.Dictionary({'A1': 0.343, 'A2': 2.21, 'η': 0.4954}),
            '0.7': ee.Dictionary({'A1': 0.254, 'A2': 2.582, 'η': 0.5911}),
            '0.8': ee.Dictionary({'A1': 0.168, 'A2': 3.094, 'η': 0.6966}),
            '0.9': ee.Dictionary({'A1': 0.084, 'A2': 3.941, 'η': 0.8191}),
            '0.95': ee.Dictionary({'A1': 0.043, 'A2': 4.840, 'η': 0.8599})
        }),
        '2': ee.Dictionary({
            '0.5': ee.Dictionary({'A1': 0.582, 'A2': 1.584, 'η': 0.2763}),
            '0.6': ee.Dictionary({'A1': 0.501, 'A2': 1.755, 'η': 0.3388}),
            '0.7': ee.Dictionary({'A1': 0.418, 'A2': 1.972, 'η': 0.4062}),
            '0.8': ee.Dictionary({'A1': 0.327, 'A2': 2.260, 'η': 0.4819}),
            '0.9': ee.Dictionary({'A1': 0.221, 'A2': 2.744, 'η': 0.5699}),
            '0.95': ee.Dictionary({'A1': 0.152, 'A2': 3.206, 'η': 0.6254})
        }),
        '3': ee.Dictionary({
            '0.5': ee.Dictionary({'A1': 0.652, 'A2': 1.458, 'η': 0.2222}),
            '0.6': ee.Dictionary({'A1': 0.580, 'A2': 1.586, 'η': 0.2736}),
            '0.7': ee.Dictionary({'A1': 0.505, 'A2': 1.751, 'η': 0.3280}),
            '0.8': ee.Dictionary({'A1': 0.419, 'A2': 1.865, 'η': 0.3892}),
            '0.9': ee.Dictionary({'A1': 0.313, 'A2': 2.320, 'η': 0.4624}),
            '0.95': ee.Dictionary({'A1': 0.238, 'A2': 2.656, 'η': 0.5084})
        }),
        '4': ee.Dictionary({
            '0.5': ee.Dictionary({'A1': 0.694, 'A2': 1.385, 'η': 0.1921}),
            '0.6': ee.Dictionary({'A1': 0.630, 'A2': 1.495, 'η': 0.2348}),
            '0.7': ee.Dictionary({'A1': 0.560, 'A2': 1.627, 'η': 0.2825}),
            '0.8': ee.Dictionary({'A1': 0.480, 'A2': 1.804, 'η': 0.3354}),
            '0.9': ee.Dictionary({'A1': 0.378, 'A2': 2.094, 'η': 0.3991}),
            '0.95': ee.Dictionary({'A1': 0.302, 'A2': 2.360, 'η': 0.4391})
        })
    })

    looksDict = ee.Dictionary(sigmaLookup.get(ee.String(looks)))
    sigmaImage = ee.Dictionary(looksDict.get(ee.String(sigma))).toImage()
    a1 = sigmaImage.select('A1')
    a2 = sigmaImage.select('A2')
    aRange = a2.subtract(a1)
    eta = sigmaImage.select('η').pow(2)

    # MMSE estimator
    mmseMask = pow.gte(a1).Or(pow.lte(a2))
    mmseIn = pow.updateMask(mmseMask)
    oneImg = ee.Image(1)
    z = mmseIn.reduceNeighborhood(ee.Reducer.mean(), kernel, None, True)
    varz = mmseIn.reduceNeighborhood(ee.Reducer.variance(), kernel)
    varx = varz.subtract(z.abs().pow(2).multiply(eta)).divide(oneImg.add(eta))
    b = varx.divide(varz)
    mmse = oneImg.subtract(b).multiply(z.abs()).add(b.multiply(mmseIn))

    # Workflow
    z99 = ee.Dictionary(pow.reduceRegion(
        reducer=ee.Reducer.percentile([99], None, 255, 0.001, 1e6),
        geometry=img.geometry(),
        scale=10,
        bestEffort=True
    )).toImage()

    overThresh = pow.gte(z99)
    K = overThresh.reduceNeighborhood(ee.Reducer.sum(), targetKernel, None, True)
    retainPixel = K.gte(Tk)
    xHat = powerToDb(pow.updateMask(retainPixel).unmask(mmse))

    return ee.Image(xHat).clip(img.geometry()).rename(bandNames)\
        .copyProperties(img, None, None)\
        .set('system:time_start', img.get('system:time_start'))


def refinedLee(image):
    bandNames = image.bandNames()
    orgImg = image
    image = dbToPower(image)

    def process_band(b):
        img = image.select([b])

        # Set up 3x3 kernels
        weights3 = ee.List.repeat(ee.List.repeat(1, 3), 3)
        kernel3 = ee.Kernel.fixed(3, 3, weights3, 1, 1, False)

        mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
        variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)

        # Use a sample of the 3x3 windows inside a 7x7 window
        sample_weights = ee.List([
            [0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 1, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 1, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 1, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 0]
        ])
        sample_kernel = ee.Kernel.fixed(7, 7, sample_weights, 3, 3, False)

        sample_mean = mean3.neighborhoodToBands(sample_kernel)
        sample_var = variance3.neighborhoodToBands(sample_kernel)

        # Determine the 4 gradients
        gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
        gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
        gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
        gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

        max_gradient = gradients.reduce(ee.Reducer.max())
        gradmask = gradients.eq(max_gradient)
        gradmask = gradmask.addBands(gradmask)

        # Determine the 8 directions
        directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(
            sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1)
        directions = directions.addBands(
            sample_mean.select(6).subtract(sample_mean.select(4)).gt(
                sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
        directions = directions.addBands(
            sample_mean.select(3).subtract(sample_mean.select(4)).gt(
                sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
        directions = directions.addBands(
            sample_mean.select(0).subtract(sample_mean.select(4)).gt(
                sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))
        directions = directions.addBands(directions.select(0).Not().multiply(5))
        directions = directions.addBands(directions.select(1).Not().multiply(6))
        directions = directions.addBands(directions.select(2).Not().multiply(7))
        directions = directions.addBands(directions.select(3).Not().multiply(8))

        directions = directions.updateMask(gradmask)
        directions = directions.reduce(ee.Reducer.sum())

        sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))
        sigmaV = sample_stats.toArray().arraySort().arraySlice(0, 0, 5).arrayReduce(ee.Reducer.mean(), [0])

        # Set up 7x7 kernels
        rect_weights = ee.List.repeat(ee.List.repeat(0, 7), 3).cat(ee.List.repeat(ee.List.repeat(1, 7), 4))
        diag_weights = ee.List([
            [1, 0, 0, 0, 0, 0, 0],
            [1, 1, 0, 0, 0, 0, 0],
            [1, 1, 1, 0, 0, 0, 0],
            [1, 1, 1, 1, 0, 0, 0],
            [1, 1, 1, 1, 1, 0, 0],
            [1, 1, 1, 1, 1, 1, 0],
            [1, 1, 1, 1, 1, 1, 1]
        ])

        rect_kernel = ee.Kernel.fixed(7, 7, rect_weights, 3, 3, False)
        diag_kernel = ee.Kernel.fixed(7, 7, diag_weights, 3, 3, False)

        dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
        dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

        for i in range(1, 4):
            dir_mean = dir_mean.addBands(
                img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2 * i + 1)))
            dir_var = dir_var.addBands(
                img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2 * i + 1)))
            dir_mean = dir_mean.addBands(
                img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2 * i + 2)))
            dir_var = dir_var.addBands(
                img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2 * i + 2)))

        dir_mean = dir_mean.reduce(ee.Reducer.sum())
        dir_var = dir_var.reduce(ee.Reducer.sum())

        varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))
        b = varX.divide(dir_var)

        return dir_mean.add(b.multiply(img.subtract(dir_mean)))\
            .arrayProject([0])\
            .arrayFlatten([['sum']])\
            .float()

    result = ee.ImageCollection(bandNames.map(process_band)).toBands().rename(bandNames)
    return powerToDb(ee.Image(result)).set('system:time_start', orgImg.get('system:time_start'))

# Export functions (not needed in Python module context, but shown for completeness)
# You can call these functions directly after defining them
# terrainCorrection = terrainCorrection
# leeSigma = leeSigma
# refinedLee = refinedLee

In [10]:
import ee, math, os
import requests
import shutil
import time
import zipfile
import rasterio
import numpy as np
from rasterio.merge import merge

ee.Initialize()

# --- Helper Functions ---

def has_data_in_roi(image, roi):
    bands = ['VV', 'VH', 'RVI']
    for band in bands:
        if band not in image.bandNames().getInfo():
            return False
        stats = image.select(band).reduceRegion(ee.Reducer.minMax(), geometry=roi, scale=10)
        min_val = stats.get(band + '_min')
        if min_val is None:
            return False
    return True

def edge_correction(image):
    """
    Applies an edge correction to remove pixels with no data along the image boundary.
    """
    edge_mask = image.mask().reduce(ee.Reducer.allNonZero())
    return image.updateMask(edge_mask).copyProperties(image, ['system:time_start'])

def get_sentinel1_collection(start_date, end_date, roi):
    """
    Loads and clips the Sentinel-1 GRD collection filtered by date, region, and polarization.
    """
    s_date = start_date.advance(-8, 'day')
    e_date = end_date.advance(8, 'day')
    
    collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                  .filterBounds(roi)
                  .filterDate(s_date, e_date)
                  .filter(ee.Filter.eq('instrumentMode', 'IW'))
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')))
    
    def process(image):
        image = image.clip(roi)
        image = edge_correction(image)
        return image

    return collection.map(process)

def calculate_rvi(image):
    """
    Calculates the Radar Vegetation Index (RVI) from the VV and VH bands.
    Formula: RVI = (4 * VH) / (VV + VH)
    """
    vv = image.select('VV')
    vh = image.select('VH')
    rvi = vh.multiply(4).divide(vv.add(vh)).rename('RVI')
    return image.addBands(rvi).set('system:time_start', image.get('system:time_start'))

def calculate_8day_composites_sar(image_collection, start_date, end_date):
    days_step = 8
    start = ee.Date(start_date)
    end = ee.Date(end_date)
    millis_step = days_step * 24 * 60 * 60 * 1000
    list_of_dates = ee.List.sequence(start.millis(), end.millis(), millis_step)

    def composite_for_millis(millis):
        composite_center = ee.Date(millis)
        composite_start = composite_center.advance(-8, 'day')
        composite_end = composite_center.advance(8, 'day')
        period_collection = image_collection.filterDate(composite_start, composite_end)
        composite = ee.Image(ee.Algorithms.If(
            period_collection.size().gt(0),
            calculate_rvi(period_collection.median()).set('system:time_start', composite_center.millis()),
            ee.Image(0).updateMask(ee.Image(0)).set('system:time_start', composite_center.millis())
        ))
        return composite

    composites = ee.ImageCollection(list_of_dates.map(composite_for_millis))
    return composites

def sort_by_time(composites):
    return composites.sort('system:time_start')

def smooth_time_series(composites):
    image_list = composites.toList(composites.size())
    collection_size = composites.size().getInfo()

    def has_rvi(img):
        return ee.Number(img.bandNames().size()).gt(3)

    smoothed_images = []
    for i in range(collection_size):
        image = ee.Image(image_list.get(i))
        image_date = ee.Date(image.get('system:time_start'))
        previous = ee.Image(image_list.get(i - 1)) if i > 0 else image
        next_img = ee.Image(image_list.get(i + 1)) if i < (collection_size - 1) else image

        current_has = has_rvi(image)
        previous_has = has_rvi(previous)
        next_has = has_rvi(next_img)

        smoothed = ee.Image(ee.Algorithms.If(
            current_has,
            ee.Image(ee.Algorithms.If(
                previous_has.And(next_has),
                ee.ImageCollection([previous, image, next_img]).mean().set('system:time_start', image_date.millis()),
                ee.Image(ee.Algorithms.If(
                    next_has,
                    ee.ImageCollection([image, next_img]).mean().set('system:time_start', image_date.millis()),
                    ee.Image(ee.Algorithms.If(
                        previous_has,
                        ee.ImageCollection([image, previous]).mean().set('system:time_start', image_date.millis()),
                        image
                    ))
                ))
            )),
            ee.Image(ee.Algorithms.If(
                previous_has.And(next_has),
                ee.ImageCollection([previous, next_img]).mean().set('system:time_start', image_date.millis()),
                ee.Image(ee.Algorithms.If(
                    previous_has,
                    previous.set('system:time_start', image_date.millis()),
                    ee.Image(ee.Algorithms.If(
                        next_has,
                        next_img.set('system:time_start', image_date.millis()),
                        image
                    ))
                ))
            ))
        ))
        smoothed_images.append(smoothed)

    return ee.ImageCollection(smoothed_images)

def download_band(image, band_name, roi, date_str, out_folder, max_retries=4):
    local_path = os.path.join(out_folder, f"{band_name}_{date_str}.tif")
    temp_dir = os.path.join(out_folder, "temp")
    os.makedirs(temp_dir, exist_ok=True)
    
    for attempt in range(max_retries):
        try:
            url = image.select([band_name]).getDownloadURL({
                'scale': 10,
                'region': roi,
                'fileFormat': 'GeoTIFF',
                'maxPixels': 1e13,
                'expires': 3600
            })
            print(f"Attempt {attempt+1} for {band_name} {date_str}: {url}")

            temp_zip_path = os.path.join(temp_dir, "download.zip")
            response = requests.get(url, stream=True, timeout=300, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)'})
            print(f"Status code: {response.status_code}")
            print(f"Response headers: {response.headers}")

            response.raise_for_status()

            with open(temp_zip_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        file.write(chunk)

            with zipfile.ZipFile(temp_zip_path, 'r') as zip_ref:
                print(f"Zip file contents: {zip_ref.namelist()}")
                zip_ref.extractall(temp_dir)

            print(f"Files in temp dir: {os.listdir(temp_dir)}")
            tif_files = [f for f in os.listdir(temp_dir) if f.endswith('.tif')]

            if len(tif_files) == 1:
                tif_file = tif_files[0]
                src_path = os.path.join(temp_dir, tif_file)
                shutil.copy(src_path, local_path)  # Use shutil.copy instead of os.replace to handle cross-device links
                if is_valid_tif(local_path):
                    print(f"Successfully downloaded {band_name} for {date_str}")
                    # Clean up temp directory after successful download
                    shutil.rmtree(temp_dir)
                    return local_path
                else:
                    print(f"⚠️ Invalid file for {band_name} {date_str}, retrying...")
                    time.sleep(5 * (attempt + 1))
            else:
                print(f"⚠️ Unexpected number of .tif files in zip: {len(tif_files)}")
                time.sleep(5 * (attempt + 1))
        except requests.exceptions.RequestException as e:
            print(f"⚠️ Request error for {band_name} {date_str}: {e}")
            time.sleep(5 * (attempt + 1))
        except Exception as e:
            print(f"Error downloading {band_name} for {date_str}: {e}")
            time.sleep(5 * (attempt + 1))
    
    print(f"Failed to download {band_name} for {date_str} after {max_retries} attempts.")
    # Clean up temp directory on failure
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    return None

def merge_bands(rvi_path, vv_path, vh_path, output_path):
    """Merge RVI, VV, VH into a single 3-band image (reshaping if necessary)."""
    band_paths = [rvi_path, vv_path, vh_path]
    bands = []

    for path in band_paths:
        with rasterio.open(path) as src:
            band = src.read(1)
            bands.append((band, src.profile))

    min_shape = min(band[0].shape for band in bands)
    bands_resized = [np.resize(band[0], min_shape) for band in bands]

    profile = bands[0][1]
    profile.update(count=3, dtype='float32')

    with rasterio.open(output_path, "w", **profile) as dst:
        for i, band in enumerate(bands_resized, start=1):
            dst.write(band, i)

    print(f"✅ Merged 3-band image saved: {output_path}")

def is_valid_tif(file_path):
    if not os.path.exists(file_path) or os.path.getsize(file_path) < 1024:
        print(f"Invalid: {file_path} - File missing or too small")
        return False
    try:
        with rasterio.open(file_path) as src:
            if src.count == 0 or src.width == 0 or src.height == 0:
                print(f"Invalid: {file_path} - No bands or zero dimensions")
                return False
            return True
    except rasterio.errors.RasterioIOError as e:
        print(f"Invalid: {file_path} - Rasterio error: {e}")
        return False

def export_sentinel1_rvi(sentinel_collection, big_folder, roi, image_name, roi_name, folder_name):
    out_folder = os.path.join(big_folder, roi_name, folder_name)
    os.makedirs(out_folder, exist_ok=True)

    image_list = sentinel_collection.toList(sentinel_collection.size())
    count = sentinel_collection.size().getInfo()

    for i in range(count):
        image = ee.Image(image_list.get(i))
        date_str = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        
        if not has_data_in_roi(image, roi):
            print(f"No data for {date_str}, skipping.")
            continue

        rvi_path = download_band(image, 'RVI', roi, date_str, out_folder)
        time.sleep(5)
        vv_path = download_band(image, 'VV', roi, date_str, out_folder)
        time.sleep(5)
        vh_path = download_band(image, 'VH', roi, date_str, out_folder)
        time.sleep(10)

        if rvi_path and vv_path and vh_path:
            output_path = os.path.join(out_folder, f"{image_name}_{date_str}.tif")
            merge_bands(rvi_path, vv_path, vh_path, output_path)
            # Remove individual band files after successful merge
            for band_path in [rvi_path, vv_path, vh_path]:
                if os.path.exists(band_path):
                    os.remove(band_path)
                    print(f"Removed individual band file: {band_path}")
        else:
            print(f"⚠️ Skipping merge for {date_str} due to missing or invalid bands.")
            # Clean up any downloaded files if merge fails
            for band_path in [rvi_path, vv_path, vh_path]:
                if band_path and os.path.exists(band_path):
                    os.remove(band_path)
                    print(f"Removed partial band file: {band_path}")

def display_rvi(rvi_collection, layer_name):
    image_list = rvi_collection.toList(rvi_collection.size())
    count = rvi_collection.size().getInfo()
    for i in range(count):
        rvi_image = ee.Image(image_list.get(i))
        if rvi_image.bandNames().contains('RVI').getInfo():
            date = ee.Date(rvi_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            print(f"{layer_name} - Image with RVI from {date}")

def export_sentinel1_rvi_drive(sentinel_collection, roi, image_name, folder_name):
    image_list = sentinel_collection.toList(sentinel_collection.size())
    count = sentinel_collection.size().getInfo()
    for i in range(count):
        image = ee.Image(image_list.get(i))
        date_str = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        task = ee.batch.Export.image.toDrive(
            image=image.select(['RVI', 'VV', 'VH']),
            description=image_name + date_str,
            folder=folder_name,
            scale=10,
            region=roi,
            fileFormat='GeoTIFF',
            maxPixels=1e13
        )
        task.start()
        print('Export initiated for image on date:', date_str)

# --- Main Execution ---

start_date = ee.Date('2023-01-01')
end_date = ee.Date('2023-02-01')
ROI = ee.Geometry.Polygon([
    [[11855180.0, 2328782.140517925], [11855180.0, 2335016.8225009246], [11851160.0, 2335016.8225009246], [11851160.0, 2328782.140517925], [11855180.0, 2328782.140517925]]
], 'EPSG:3857')

big_folder = "/mnt/data1tb/LSTRetrieval/Code/download_data"

sentinel1_collection = get_sentinel1_collection(start_date, end_date, ROI)
print('Sentinel-1 collection size:', sentinel1_collection.size().getInfo())

sentinel_8day_composites = calculate_8day_composites_sar(sentinel1_collection, start_date, end_date)
print('8-day composites size:', sentinel_8day_composites.size().getInfo())

sorted_composites = sort_by_time(sentinel_8day_composites)
print('First composite band names:', sorted_composites.first().bandNames().getInfo())

smoothed_composites = smooth_time_series(sorted_composites)

export_sentinel1_rvi(smoothed_composites, big_folder, ROI, 'rvi_8days', "BinhThanh_DucHue_LongAn", 'BinhThanh_rvi_8days')
# export_sentinel1_rvi_drive(smoothed_composites, ROI, 'rvi_8days_', 'LST')

Sentinel-1 collection size: 12
8-day composites size: 4
First composite band names: ['VV', 'VH', 'angle', 'RVI']
Attempt 1 for RVI 2023-01-01: https://earthengine.googleapis.com/v1/projects/172099900522/thumbnails/aa785a16fdbb411fbfd6e7aea15f57c1-b8fa0e99df935855b3f2adf524b90263:getPixels
Status code: 200
Response headers: {'Cache-Control': 'private, max-age=3600', 'Expires': 'Tue, 11 Mar 2025 05:33:28 GMT', 'Content-Type': 'application/zip', 'Content-disposition': 'attachment; filename=download.zip', 'Vary': 'Origin, X-Origin, Referer', 'Date': 'Tue, 11 Mar 2025 04:33:28 GMT', 'Server': 'ESF', 'Content-Length': '1732378', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}
Zip file contents: ['download.RVI.tif']
Files in temp dir: ['download.zip', 'download.RVI.tif']
Successfully downloaded RVI for 2023-01-01
Attempt 1 for VV 2023-01-01: https://earthengine.googleapis.com/v1/project